In [ ]:
import pandas as pd
import numpy as np
import os
import optuna

from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import TargetEncoder
from sklearn.pipeline import Pipeline
from sklearn.base import clone
from sklearn.model_selection import TimeSeriesSplit, cross_validate, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge

import json
from scipy.optimize import minimize
from sklearn.metrics import root_mean_squared_error

In [ ]:
# 원본 train 데이터
df = pd.read_csv('train_call.csv', encoding = 'cp949')

# 결측치 명시
df = df.replace(-99.0, None)

# 컬럼명 정리
df.columns = df.columns.str.replace('^call119_train\\.', '', regex = True)

# 데이터 추가

### 구별 이전 달의 주민등록인구

In [ ]:
# 'address_gu' 별 주민등록인구: KOSIS > 지역통계 > 인구 및 사회 (사회조사 외) > 부산광역시 > 부산광역시주민등록인구통계 > 주민등록인구총괄 > 부산광역시 전체 세대 및 인구개황
gu_pop = pd.read_csv('gu_pop.csv', encoding = 'cp949', dtype = {'시점': str}, na_values = '-')

# 컬럼명 정리
gu_pop = gu_pop.rename(
    columns = {'시점': 'tm',
               '구·군별': 'address_gu',
               '읍면동수 (개)': 'sub_address_count',
               '세대수 (세대)': 'gu_household',
               '인구수  (명)': 'gu_pop',
               '남자인구수 (명)': 'gu_male_pop',
               '여자인구수 (명)': 'gu_female_pop',
               '시전체 인구에 대한 구성비 (%)': 'gu_pop_ratio',
               '면적 (㎢)': 'gu_area',
               '인구밀도 (명/㎢)': 'gu_density'}
)

# 날짜 컬럼 정리
gu_pop['tm'] = pd.to_datetime(gu_pop['tm'].astype(str), format = '%Y.%m')
gu_pop['year'] = gu_pop['tm'].dt.year.astype(int)
gu_pop['month'] = gu_pop['tm'].dt.month.astype(int)
gu_pop = gu_pop.drop('tm', axis = 1)

# 불필요한 날짜 제거
gu_pop = gu_pop[gu_pop['month'].isin(range(4, 10))].reset_index(drop = True)

# 이전 달의 정보를 현재의 feature로 이동
gu_pop['month'] = gu_pop['month'] + 1

# 회의 후 drop
gu_pop = gu_pop.drop(
    ['sub_address_count', 'gu_household', 'gu_pop', 'gu_male_pop', 'gu_female_pop'],
    axis = 1
)

# 저장
gu_pop.to_csv('merge0.csv', index = False, encoding = 'cp949')

# 조회
gu_pop

### 읍면동별 이전 달의 주민등록인구

In [ ]:
# --------------------------------------------------
# 'sub_address' 별 주민등록인구: KOSIS > 지역통계 > 인구 및 사회 (사회조사 외) > 부산광역시 > 부산광역시주민등록인구통계 > 주민등록인구총괄 > 구·군 및 읍·면·동 세대와 인구
# --------------------------------------------------
# 행정동 변경으로 인해, 일부 지역은 평균치로 대체함
# --------------------------------------------------
sub_pop = pd.read_csv('sub_pop.csv', encoding = 'cp949', dtype = {'시점': str}, na_values = '-')

# 불필요한 컬럼 삭제
sub_pop = sub_pop.drop(['내외국인별', 'Unnamed: 7'], axis = 1)


# 컬럼명 정리
sub_pop = sub_pop.rename(
    columns = {'구·군별': 'sub_address',
               '시점': 'tm',
               '세대수[세대]': 'sub_household',
               '인구[명]': 'sub_pop',
               '남자인구[명]': 'sub_male_pop',
               '여자인구[명]': 'sub_female_pop'}
)

# 'address_gu' 컬럼 추가 (송정동은 강서구와 해운대구에 모두 존재)
gu_set = set(df['address_gu'])
sub_pop['address_gu'] = sub_pop['sub_address'].where(
    sub_pop['sub_address'].isin(gu_set)
)
sub_pop['address_gu'] = sub_pop['address_gu'].ffill()

# 날짜 컬럼 정리
sub_pop['tm'] = sub_pop['tm'].str.replace(
    r'\s[가-힣]+', '', regex = True
)
sub_pop['tm'] = pd.to_datetime(sub_pop['tm'].astype(str), format = '%Y.%m')
sub_pop['year'] = sub_pop['tm'].dt.year.astype(int)
sub_pop['month'] = sub_pop['tm'].dt.month.astype(int)
sub_pop = sub_pop.drop('tm', axis = 1)

# 불필요한 날짜 제거
sub_pop = sub_pop[sub_pop['month'].isin(range(4, 10))].reset_index(drop = True)

# 이전 달의 정보를 현재의 feature로 이동
sub_pop['month'] = sub_pop['month'] + 1

# 'sub_address'가 'address_gu'인 자료 제거
sub_pop = sub_pop[~sub_pop['sub_address'].isin(gu_set)]



# --------------------------------------------------
# 'sub_address' 별 1인 가구 수: KOSIS > 지역통계 > 인구 및 사회 (사회조사 외) > 부산광역시 > 부산광역시주민등록인구통계 > 주민등록인구총괄 > 읍·면·동별 세대원수별 세대수
# --------------------------------------------------
# 행정동 변경으로 인한 결측치를 함께 대체하기 위해 통합
# --------------------------------------------------
sub_single = pd.read_csv('sub_single.csv', encoding = 'cp949', dtype = {'시점': str}, na_values = '-')

# 컬럼명 정리
sub_single = sub_single.rename(
    columns = {'시점': 'tm',
               '구·군별(1)': 'address_gu',
               '구·군별(2)': 'sub_address',
               '1인': 'sub_single'}
)

# 날짜 컬럼 정리
sub_single['tm'] = pd.to_datetime(sub_single['tm'].astype(str), format = '%Y.%m')
sub_single['year'] = sub_single['tm'].dt.year.astype(int)
sub_single['month'] = sub_single['tm'].dt.month.astype(int)
sub_single = sub_single.drop('tm', axis = 1)

# 불필요한 날짜 제거
sub_single = sub_single[sub_single['month'].isin(range(4, 10))].reset_index(drop = True)

# 이전 달의 정보를 현재의 feature로 이동
sub_single['month'] = sub_single['month'] + 1

# --------------------------------------------------
# 통합
# --------------------------------------------------
sub_pop = sub_pop.merge(sub_single, how = 'left', on = ['year', 'month', 'address_gu', 'sub_address'])

# --------------------------------------------------
# 행정동 변화가 없는 'sub_address'
# --------------------------------------------------
# ! ! !단, '일광면'과 '정관면'은 train 및 test 데이터를 수정 ! ! !
# --------------------------------------------------
A0 = sub_pop[
    ~sub_pop['sub_address']
    .isin([
        '중앙동', '영주1동', '영주2동', '광복동',
        '부민동', '충무동', '남항동', '부전1동',
        '부전2동', '수민동', '복산동', '반송1동',
        '반송2동', '금사회동동', '청룡노포동', '선두구동',
        '부곡1동', '부곡2동', '부곡3동', '부곡4동',
        '녹산동', '송정동', '가덕도동', '가락동',
        '대저1동', '대저2동'
    ])
]
A0 = A0.copy()
A0['sub_address'] = A0['sub_address'].str.replace(
    '[0-9]+(?=동)', '', regex = True
)
A0 = A0.groupby(['year', 'month', 'address_gu', 'sub_address']).sum().reset_index()
A1 = sub_pop[
    sub_pop['sub_address']
    .isin(['대저1동', '대저2동'])
]
A = pd.concat([A0, A1], ignore_index = True)

# --------------------------------------------------
# 행정동 변화: 대창동 <- (중앙동, 영주동) / 중앙동 <- 중앙동 / (대창동, 영주동) <- 영주동
# --------------------------------------------------
B0 = sub_pop[
    sub_pop['sub_address']
    .isin(['중앙동', '영주1동', '영주2동'])
]

B0 = B0.copy()
B0['sub_address'] = B0['sub_address'].str.replace(
    '[0-9]+(?=동)', '', regex = True
)
B0 = B0.groupby(['year', 'month', 'address_gu', 'sub_address']).sum().reset_index()

# 대창동
B1 = B0.copy()
B1 = B1.groupby(['year', 'month', 'address_gu'])[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]].mean().reset_index()
B1['sub_address'] = '대창동'

# 중앙동
B2 = B0.copy()
B2 = B2.loc[B2['sub_address'] == '중앙동', :] 
B2[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] = B2[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] / 2

# 영주동
B3 = B0.copy()
B3 = B3.loc[B3['sub_address'] == '영주동', :] 
B3[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] = B3[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] / 2

# 통합
B = pd.concat([B1, B2, B3], ignore_index = True)

# --------------------------------------------------
# 행정동 변화: (신창동, 광복동, 창선동) <- 광복동
# --------------------------------------------------
C0 = sub_pop[
    sub_pop['sub_address']
    .isin(['광복동'])
]

# 광복동
C1 = C0.copy() 
C1[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] = C1[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] / 3

# 신창동
C2 = C1.copy() 
C2['sub_address'] = '신창동'

# 창선동
C3 = C1.copy() 
C3['sub_address'] = '창선동'

# 통합
C = pd.concat([C1, C2, C3], ignore_index = True)

# --------------------------------------------------
# 행정동 변화: (부민동, 부용동) <- 부민동
# --------------------------------------------------
D0 = sub_pop[
    sub_pop['sub_address']
    .isin(['부민동'])
]

# 부민동
D1 = D0.copy()
D1[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] = D1[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] / 2

# 부용동
D2 = D1.copy()
D2['sub_address'] = '부용동'

# 통합
D = pd.concat([D1, D2], ignore_index = True)

# --------------------------------------------------
# 행정동 변화: (충무동, 토성동) <- 충무동
# --------------------------------------------------
E0 = sub_pop[
    sub_pop['sub_address']
    .isin(['충무동'])
]

# 충무동
E1 = E0.copy()
E1[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] = E1[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] / 2

# 토성동
E2 = E1.copy()
E2['sub_address'] = '토성동'

# 통합
E = pd.concat([E1, E2], ignore_index = True)

# --------------------------------------------------
# 행정동 변화: (남항동, 대교동, 대평동) <- 남항동
# --------------------------------------------------
F0 = sub_pop[
    sub_pop['sub_address']
    .isin(['남항동'])
]

# 남항동
F1 = F0.copy()
F1[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] = F1[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] / 3

# 대교동
F2 = F1.copy()
F2['sub_address'] = '대교동'

# 대평동
F3 = F1.copy()
F3['sub_address'] = '대평동'

# 통합
F = pd.concat([F1, F2, F3], ignore_index = True)

# 저장
#sub_pop.to_csv('merge1.csv', index = False, encoding = 'cp949')

# --------------------------------------------------
# 행정동 변화: (부전동, 범전동) <- 부전1동, 부전동 <- 부전2동
# --------------------------------------------------
G0 = sub_pop[
    sub_pop['sub_address']
    .isin(['부전1동', '부전2동'])
]

# 부전동
G1 = G0.copy()
G1 = G1.groupby(['year', 'month', 'address_gu'])[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]].mean().reset_index()
G1['sub_address'] = '부전동'

# 범전동
G2 = sub_pop[
    sub_pop['sub_address']
    .isin(['부전1동'])
]
G2 = G2.copy()
G2[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] = G2[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] / 2
G2['sub_address'] = '범전동'

# 통합
G = pd.concat([G1, G2], ignore_index = True)

# --------------------------------------------------
# 행정동 변화: (낙민동, 수안동) <- 수민동
# --------------------------------------------------
H0 = sub_pop[
    sub_pop['sub_address']
    .isin(['수민동'])
]

# 낙민동
H1 = H0.copy()
H1[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] = H1[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] / 2
H1['sub_address'] = '낙민동'

# 수안동
H2 = H1.copy()
H2['sub_address'] = '수안동'

# 통합
H = pd.concat([H1, H2], ignore_index = True)

# --------------------------------------------------
# 행정동 변화: (복천동, 칠산동) <- 복산동
# --------------------------------------------------
I0 = sub_pop[
    sub_pop['sub_address']
    .isin(['복산동'])
]

# 복천동
I1 = I0.copy()
I1[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] = I1[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] / 2
I1['sub_address'] = '복천동'

# 칠산동
I2 = I1.copy()
I2['sub_address'] = '칠산동'

# 통합
I = pd.concat([I1, I2], ignore_index = True)

# --------------------------------------------------
# 행정동 변화: (반송동, 석대동) <- 반송1동, 반송동 <- 반송2동
# --------------------------------------------------
J0 = sub_pop[
    sub_pop['sub_address']
    .isin(['반송1동', '반송2동'])
]

# 반송동
J1 = J0.copy()
J1 = J1.groupby(['year', 'month', 'address_gu'])[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]].mean().reset_index()
J1['sub_address'] = '반송동'

# 석대동
J2 = sub_pop[
    sub_pop['sub_address']
    .isin(['반송1동'])
]
J2 = J2.copy()
J2[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] = J2[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] / 2
J2['sub_address'] = '석대동'

# 통합
J = pd.concat([J1, J2], ignore_index = True)

# --------------------------------------------------
# 행정동 변화: (금사동, 회동동) <- 금사회동동
# --------------------------------------------------
K0 = sub_pop[
    sub_pop['sub_address']
    .isin(['금사회동동'])
]

# 금사동
K1 = K0.copy()
K1[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] = K1[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] / 2
K1['sub_address'] = '금사동'

# 회동동
K2 = K1.copy()
K2['sub_address'] = '회동동'

# 통합
K = pd.concat([K1, K2], ignore_index = True)

# --------------------------------------------------
# 행정동 변화: (청룡동, 노포동) <- 청룡노포동
# --------------------------------------------------
L0 = sub_pop[
    sub_pop['sub_address']
    .isin(['청룡노포동'])
]

# 청룡동
L1 = L0.copy()
L1[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] = L1[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] / 2
L1['sub_address'] = '청룡동'

# 노포동
L2 = L1.copy()
L2['sub_address'] = '노포동'

# 통합
L = pd.concat([L1, L2], ignore_index = True)

# --------------------------------------------------
# 행정동 변화: (선동, 두구동) <- 선두구동
# --------------------------------------------------
M0 = sub_pop[
    sub_pop['sub_address']
    .isin(['선두구동'])
]

# 선동
M1 = M0.copy()
M1[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] = M1[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] / 2
M1['sub_address'] = '선동'

# 두구동
M2 = M1.copy()
M2['sub_address'] = '두구동'

# 통합
M = pd.concat([M1, M2], ignore_index = True)

# --------------------------------------------------
# 행정동 변화: 오륜동 <- 부곡3동, 부곡동 <- (부곡1동, 부곡2동, 부곡3동, 부곡4동)
# --------------------------------------------------
# 오륜동
N1 = sub_pop[
    sub_pop['sub_address']
    .isin(['부곡3동'])
]
N1 = N1.copy()
N1[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] = N1[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] / 2
N1['sub_address'] = '오륜동'

# 부곡동
N2 = sub_pop[
    sub_pop['sub_address']
    .isin(['부곡1동', '부곡2동', '부곡4동'])
]
N2 = N2.copy()
N2 = pd.concat([N1, N2])
N2 = N2.groupby(['year', 'month', 'address_gu'])[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]].sum().reset_index()
N2['sub_address'] = '부곡동'

# 통합
N = pd.concat([N1, N2], ignore_index = True)

# --------------------------------------------------
# 행정동 변화: (녹산동, 구랑동, 미음동, 범방동, 생곡동, 화전동, 지사동, 신호동, 송정동) <- 녹산동, 송정동 <- 송정동
# --------------------------------------------------
# ! ! ! 강서구의 송정동 (해운대에도 송정동이 있음) ! ! !
# --------------------------------------------------
# 녹산동
O1 = sub_pop[
    sub_pop['sub_address']
    .isin(['녹산동'])
]
O1 = O1.copy()
O1[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] = O1[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] / 9

# 구랑동, 미음동, 범방동, 생곡동, 화전동, 지사동, 신호동
O2 = O1.copy()
O2['sub_address'] = '구랑동'
O3 = O1.copy()
O3['sub_address'] = '미음동'
O4 = O1.copy()
O4['sub_address'] = '범방동'
O5 = O1.copy()
O5['sub_address'] = '생곡동'
O6 = O1.copy()
O6['sub_address'] = '화전동'
O7 = O1.copy()
O7['sub_address'] = '지사동'
O8 = O1.copy()
O8['sub_address'] = '신호동'

# 강서구 송정동
O9 = O1.copy()
O9['sub_address'] = '송정동'

# 통합
O = pd.concat([O1, O2, O3, O4, O5, O6, O7, O8, O9], ignore_index = True)

# --------------------------------------------------
# 해운대구 송정동
# --------------------------------------------------
P = sub_pop[
    (sub_pop['sub_address']
    .isin(['송정동'])) &
    (sub_pop['address_gu'] == '해운대구')
]
P = P.reset_index(drop = True)

# --------------------------------------------------
# 행정동 변화: (눌차동, 대항동, 동선동, 성북동, 천성동) <- 가덕도동
# --------------------------------------------------
Q0 = sub_pop[
    sub_pop['sub_address']
    .isin(['가덕도동'])
]

# 눌차동
Q1 = Q0.copy()
Q1[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] = Q1[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] / 5
Q1['sub_address'] = '눌차동'

# 대항동, 동선동, 성북동, 천성동
Q2 = Q1.copy()
Q2['sub_address'] = '대항동'
Q3 = Q1.copy()
Q3['sub_address'] = '동선동'
Q4 = Q1.copy()
Q4['sub_address'] = '성북동'
Q5 = Q1.copy()
Q5['sub_address'] = '천성동'

# 통합
Q = pd.concat([Q1, Q2, Q3, Q4, Q5], ignore_index = True)

# --------------------------------------------------
# 행정동 변화: (봉림동, 식만동, 죽동동, 죽림동) <- 가락동
# --------------------------------------------------
R0 = sub_pop[
    sub_pop['sub_address']
    .isin(['가락동'])
]

# 봉림동
R1 = R0.copy()
R1[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] = R1[[
    'sub_household', 'sub_pop', 'sub_male_pop', 'sub_female_pop', 'sub_single'
]] / 4
R1['sub_address'] = '봉림동'

# 식만동, 죽동동, 죽림동
R2 = R1.copy()
R2['sub_address'] = '식만동'
R3 = R1.copy()
R3['sub_address'] = '죽동동'
R4 = R1.copy()
R4['sub_address'] = '죽림동'

# 통합
R = pd.concat([R1, R2, R3, R4], ignore_index = True)

# --------------------------------------------------
# 통합
# --------------------------------------------------
sub_pop_imputed = pd.concat([A, B, C, D, E, F, G, H, I, J, K, L, M, N, O, P, Q, R], ignore_index = True)

# --------------------------------------------------
# 저장
# --------------------------------------------------
sub_pop_imputed.to_csv('merge1.csv', index = False, encoding = 'cp949')

# --------------------------------------------------
# 조회
# --------------------------------------------------
sub_pop_imputed

### 구별 이전 달의 고령인구

In [ ]:
# 'address_gu' 별 65세 이상 고령인구: KOSIS > 지역통계 > 인구 및 사회 (사회조사 외) > 부산광역시 > 부산광역시주민등록인구통계 > 구·군별 연령별 현황 > 구·군별 연령별(5세) 인구
gu_old = pd.read_csv('gu_old.csv', encoding = 'cp949', dtype = {'시점': str}, na_values = '-')

# 불필요한 컬럼 삭제
gu_old = gu_old.drop('연령별(1)', axis = 1)

# 컬럼명 정리
gu_old = gu_old.rename(
    columns = {'시점': 'tm',
               '구군별(1)': 'address_gu',
               '계': 'gu_old',
               '남자': 'gu_male_old',
               '여자': 'gu_female_old'}
)

# 날짜 컬럼 정리
gu_old['tm'] = pd.to_datetime(gu_old['tm'].astype(str), format = '%Y.%m')
gu_old['year'] = gu_old['tm'].dt.year.astype(int)
gu_old['month'] = gu_old['tm'].dt.month.astype(int)
gu_old = gu_old.drop('tm', axis = 1)

# 불필요한 날짜 제거
gu_old = gu_old[gu_old['month'].isin(range(4, 10))].reset_index(drop = True)

# 이전 달의 정보를 현재의 feature로 이동
gu_old['month'] = gu_old['month'] + 1

# 저장
gu_old.to_csv('merge2.csv', index = False, encoding = 'cp949')

# 조회
gu_old

### 신고가 일어난 가장 최근 날까지의 신고 카테고리별 누적 건수 및 비율

In [ ]:
# --------------------------------------------------
# 신고가 일어난 가장 최근 날까지의 신고 카테고리별 누적 건수 및 비율: 'train_cat.csv', 'test_cat119.csv'
# --------------------------------------------------
# ! ! ! 2020년은 NaN이 너무 많아져서 그냥 해당 년도의 자료를 사용 ! ! !
# --------------------------------------------------
# ! ! ! 'call_count'가 없는 2024년은 모두 2020-2023년의 median으로 대체 ! ! ! 
# --------------------------------------------------
cat1 = pd.read_csv('train_cat.csv', encoding = 'cp949')
cat2 = pd.read_csv('test_cat119.csv', encoding = 'cp949')

# 컬럼명 정리
cat1.columns = cat1.columns.str.replace('^cat119_train\\.', '', regex = True)
cat2 = cat2.rename(
    columns = {'TM': 'tm', 'STN': 'stn'}
)

# 불필요한 컬럼 정리
cat1 = cat1.drop(['Unnamed: 0', 'address_city', 'stn'], axis = 1)
cat2 = cat2.drop(['address_city', 'stn'], axis = 1)

# 2024년 'sub_cat' 별 'call_count' imputation
# ! ! ! 2020-2023년에는 기타>상황출동만 존재하지만, 2024년에는 구조>상황출동이 하나 존재함. 'cat' 무시하고 기타>상황출동의 값으로 imputation ! ! !
impute2024 = cat1.groupby(['sub_cat'])['call_count'].median().reset_index()
cat2 = cat2.merge(impute2024, how = 'left', on = ['sub_cat'])

# 2020-2024 통합
cat = pd.concat([cat1, cat2], ignore_index = True)

# 날짜 컬럼 정리
cat['tm'] = pd.to_datetime(cat['tm'].astype(str), format = '%Y%m%d')
cat['tm'] = cat['tm'] + pd.Timedelta(days = 1) # (1) 대희 딸깎
cat['year'] = cat['tm'].dt.year.astype(int)
cat['month'] = cat['tm'].dt.month.astype(int)
cat['day'] = cat['tm'].dt.day.astype(int)
cat = cat.drop('tm', axis = 1)

# 행정구 통일
cat['sub_address'] = cat['sub_address'].replace({'일광면': '일광읍', '정관면': '정관읍'})

In [ ]:
# --------------------------------------------------
# 'cat' 비율 및 건수 계산
# --------------------------------------------------
# 'cat'별 'call_count' 합계
pivot_cat = cat.copy().groupby([
    'year', 'month', 'day', 'address_gu', 'sub_address', 'cat'
])['call_count'].sum().reset_index()

# 'cat'을 컬럼으로 pivot
pivot_cat = pivot_cat.pivot_table(
    index = ['year', 'month', 'day', 'address_gu', 'sub_address'],
    columns = 'cat',
    values = 'call_count',
    fill_value = 0
).reset_index()

# 'call_sum' (날짜 + 지역 별 총 'call _count') 계산
cat_cols = pivot_cat.columns.difference(
    ['year', 'month', 'day', 'address_gu', 'sub_address']
).to_list()
pivot_cat['call_sum'] = pivot_cat[cat_cols].sum(axis = 1)

# 날짜 범위 만들기
date_range = pd.date_range(start = '2020-05-01', end = '2024-10-31', freq='D')
date_df = pd.DataFrame({
    'year': date_range.year,
    'month': date_range.month,
    'day': date_range.day
})

# Unique 'address_gu' + 'sub_address' 조합 추출
sub_addrs = pivot_cat[['address_gu', 'sub_address']].drop_duplicates()
sub_addrs = sub_addrs.sort_values(['address_gu', 'sub_address']).reset_index(drop = True)

# 'year' + 'month' + 'day' + 'address_gu' + 'sub_address' 조합 생성
full_index = date_df.merge(sub_addrs, how = 'cross')

# 생성된 모든 조합과 'pivot_cat'을 병합
pivot_cat_filled = full_index.merge(
    pivot_cat, 
    on = ['year', 'month', 'day', 'address_gu', 'sub_address'],
    how = 'left'
)

# 빈 날짜 + 지역의 신고 건수 결측치를 0으로 설정
pivot_cat_filled.fillna(0, inplace = True)

# 정렬
pivot_cat_filled = pivot_cat_filled.sort_values([
    'address_gu', 'sub_address', 'year', 'month', 'day'
]).reset_index(drop = True)

# 'call_sum' (누적 'call _count') 누적합 계산
pivot_cat_filled['cum_call_sum'] = pivot_cat_filled.groupby([
    'address_gu', 'sub_address' # (2) 대희야 여기 'address_gu'가 빠져서 송정동 ratio 합이 1이 안되서 추가했다
])['call_sum'].cumsum()

# 'cat' 별 'call_sum' 계산
for col in cat_cols:
    cum_col = f'cum_{col}'
    pivot_cat_filled[cum_col] = pivot_cat_filled.groupby([
        'address_gu', 'sub_address'
    ])[col].cumsum()

# 누적 비율 계산
for col in cat_cols:
    pivot_cat_filled[f'cum_{col}_ratio'] = pivot_cat_filled[f'cum_{col}'] / pivot_cat_filled['cum_call_sum']

# 비율 결측치를 0으로 설정
pivot_cat_filled.fillna(0, inplace = True)

In [ ]:
# --------------------------------------------------
# 'sub_cat' 비율 및 건수 계산
# --------------------------------------------------
# 'sub_cat'별 'call_count' 합계
pivot_subcat = cat.copy().groupby([
    'year', 'month', 'day', 'address_gu', 'sub_address', 'sub_cat'
])['call_count'].sum().reset_index()

# 'sub_cat'을 컬럼으로 pivot
pivot_subcat = pivot_subcat.pivot_table(
    index=['year', 'month', 'day', 'address_gu', 'sub_address'],
    columns='sub_cat',
    values='call_count',
    fill_value=0
).reset_index()

# 'call_sum' (날짜 + 지역 별 총 'call _count') 계산
subcat_cols = pivot_subcat.columns.difference(
    ['year', 'month', 'day', 'address_gu', 'sub_address']
).to_list()
pivot_subcat['call_sum'] = pivot_subcat[subcat_cols].sum(axis = 1)

# 'year' + 'month' + 'day' + 'address_gu' + 'sub_address' 조합과 'pivot_subcat'을 병합
pivot_subcat_filled = full_index.merge(
    pivot_subcat, 
    on = ['year', 'month', 'day', 'address_gu', 'sub_address'],
    how = 'left'
)

# 빈 날짜 + 지역의 신고 건수 결측치를 0으로 설정
pivot_subcat_filled.fillna(0, inplace = True)

# 정렬
pivot_subcat_filled = pivot_subcat_filled.sort_values([
    'address_gu', 'sub_address', 'year', 'month', 'day'
]).reset_index(drop = True)

# 'call_sum' (누적 'call _count') 누적합 계산
pivot_subcat_filled['cum_call_sum'] = pivot_subcat_filled.groupby([
    'address_gu', 'sub_address'
])['call_sum'].cumsum()

# 'sub_cat'별 'call_sum' 계산
for col in subcat_cols:
    cum_col = f'cum_{col}'
    pivot_subcat_filled[cum_col] = pivot_subcat_filled.groupby([
        'address_gu', 'sub_address'
    ])[col].cumsum()

# 누적 비율 계산
for col in subcat_cols:
    pivot_subcat_filled[f'cum_{col}_ratio'] = pivot_subcat_filled[f'cum_{col}'] / pivot_subcat_filled['cum_call_sum']

# 비율 결측치를 0으로 설정
pivot_subcat_filled.fillna(0, inplace = True)

In [ ]:
# --------------------------------------------------
# 병합
# --------------------------------------------------
# 혹시 모르니 동일한 순서로 정렬
sort_keys = ['year', 'month', 'day', 'address_gu', 'sub_address']
pivot_cat_filled = pivot_cat_filled.sort_values(by = sort_keys).reset_index(drop = True)
pivot_subcat_filled = pivot_subcat_filled.sort_values(by = sort_keys).reset_index(drop = True)

# 키 컬럼 중복 방지를 위해 'pivot_subcat_filled'에서 키 컬럼, 중복 컬럼 제외
pivot_subcat_filled_only = pivot_subcat_filled.drop(
    columns = ['year', 'month', 'day', 'address_gu', 'sub_address', 'call_sum', 'cum_call_sum'] # (3) 대희 같은 컬럼이 더 있었습니다
)

# 중복된 컬럼명 변경
pivot_subcat_filled_only = pivot_subcat_filled_only.rename(
    columns = {'cum_기타': 'cum_sub_기타',
               'cum_기타_ratio': 'cum_sub_기타_ratio',
               '기타': 'cum_sub_기타_ratio'}
) # (4) 대희 합치니까 이상한 suffix가 생기길래 확인해 보니 이름이 같은 컬럼들이 있었습니다

# 병합
pivot_df = pd.concat([pivot_cat_filled, pivot_subcat_filled_only], axis = 1)

# 불필요한 날짜 삭제
pivot_df = pivot_df[pivot_df['month'].isin(range(5, 11))].reset_index(drop = True)

# --------------------------------------------------
# 휘의 후 필요한 컬럼만 선택: 구급 기타만 'sub_cat'에서 가져감
# --------------------------------------------------
pivot_df = pivot_df.loc[:, pivot_cat_filled.columns.to_list() + ['구급기타', 'cum_구급기타', 'cum_구급기타_ratio']]
pivot_df = pivot_df.copy()

# --------------------------------------------------
# 저장
# --------------------------------------------------
pivot_df.to_csv('merge4.csv', index = False, encoding = 'cp949') 

# --------------------------------------------------
# 조회
# --------------------------------------------------
pivot_df

### 구별 위경도와 지역 클러스터링

In [ ]:
# 'address_gu'의 위경도: 국토부 브이월드 > 공간정보 다운로드 > 행정구역시군구_경계 > 위경도 추출
gu_lat_lon = pd.read_csv("gu_lat_lon.csv", encoding = 'cp949')

# address_gu 만들고 남은거 drop
gu_lat_lon['address_gu'] = gu_lat_lon['SGG_NM'].str.replace('부산광역시 ', '', regex = False)
gu_lat_lon = gu_lat_lon.drop(columns = ['ADM_SECT_C', 'SGG_NM', 'SGG_OID', 'COL_ADM_SE'])

# 클러스터 추가(Elbow, Silhouette 참고 => 4개 설정)
gu_lat_lon['gu_cluster'] = KMeans(n_clusters = 4, random_state = 42).fit_predict(
    gu_lat_lon[['gu_lat', 'gu_lon']]
)
gu_lat_lon['gu_cluster'] = gu_lat_lon['gu_cluster'].astype(object)

# 저장
gu_lat_lon.to_csv('merge6.csv', index = False, encoding = 'cp949')

# 조회
gu_lat_lon

### 읍면동별 위경도

In [ ]:
# 국토부 브이월드 > 공간정보 다운로드 > 행정구역_읍면동(법정동) > QGIS 정제 > 'sub_address'의 위도 및 경도 추출
sub_lat_lon = pd.read_csv("sub_lat_lon.csv", encoding = 'cp949')

# address_gu, sub_address로 groupby해서 위경도 평균 구하기
sub_lat_lon = sub_lat_lon.groupby([
    'address_gu', 'sub_address'
])[[
    'latitude', 'longitude'
]].mean().reset_index()

# 컬럼명 변경(sub_lat, sub_lon)
sub_lat_lon = sub_lat_lon.rename(columns = {
    'latitude': 'sub_lat',
    'longitude': 'sub_lon'
})

# 저장
sub_lat_lon.to_csv('merge7.csv', index = False, encoding = 'cp949')

# 조회
sub_lat_lon

### 읍면동 별 전년도의 신고 건수 중간값

In [ ]:
py = df.copy()[['tm', 'address_gu', 'sub_address', 'call_count']]
py['sub_address'] = py['sub_address'].replace({'일광면': '일광읍', '정관면': '정관읍'})

# 날짜 컬럼 정리
py['tm'] = pd.to_datetime(py['tm'].astype(str), format = '%Y%m%d')
py['year'] = py['tm'].dt.year.astype(int)
py['month'] = py['tm'].dt.month.astype(int)

# 날짜 범위
date_range = pd.date_range(start = '2020-05-01', end = '2024-10-31', freq='ME')
date_df = pd.DataFrame({
    'year': date_range.year,
    'month': date_range.month
})

# Unique 'address_gu' + 'sub_address' 조합 추출
sub_addrs = py[['address_gu', 'sub_address']].drop_duplicates()
sub_addrs = sub_addrs.sort_values(['address_gu', 'sub_address']).reset_index(drop = True)

# 'year' + 'month' + 'day' + 'address_gu' + 'sub_address' 조합 생성
full_index = date_df.merge(sub_addrs, how = 'cross')
full_index = full_index.loc[full_index['month'].isin(range(5, 11))]

# 금년 중간값
prev_year_call_med = py.groupby(
    ['year', 'month', 'address_gu', 'sub_address']
)['call_count'].median().reset_index()

# 작년 중앙값
prev_year_call_med['year'] += 1

# 병합
prev_year_call_med = full_index.merge(
    prev_year_call_med,
    how = 'left',
    on = ['year', 'month', 'address_gu', 'sub_address']
)

# 결측값은 연도에 무관한 월별 중간값으로 대체
prev_year_call_med['call_count'] = prev_year_call_med['call_count'].fillna(
    prev_year_call_med.groupby(['month', 'address_gu', 'sub_address'])['call_count'].transform('median')
)

# 대체하고도 사건이 일어나지 않아 결측치인 값은 0으로 대체
prev_year_call_med.fillna(0, inplace = True)

# 컬럼명 정리
prev_year_call_med = prev_year_call_med.rename(
    columns = {'call_count': 'prev_year_call_med'}
)

# 저장
prev_year_call_med.to_csv('merge8.csv', index = False, encoding = 'cp949')

# 조회
prev_year_call_med

### 읍면동 별 전년도 월별 신고 카테고리 비율

In [ ]:
# --------------------------------------------------
# 전년도 월별 신고 카테고리별 누적 건수 및 비율: 'train_cat.csv'
# --------------------------------------------------
# ! ! ! 2020년은 NaN이 너무 많아져서 그냥 해당 년도의 자료를 사용 ! ! !
# --------------------------------------------------
# 바로 전 'year'의 'sub_address' 별 'cat' 비율
call_ratio = pd.read_csv('train_cat.csv', encoding = 'cp949')

# 컬럼명 정리
call_ratio.columns = call_ratio.columns.str.replace('^cat119_train\\.', '', regex = True)

# 날짜 컬럼 정리
call_ratio['tm'] = pd.to_datetime(call_ratio['tm'].astype(str), format = '%Y%m%d')
call_ratio['year'] = call_ratio['tm'].dt.year.astype(int)
call_ratio['month'] = call_ratio['tm'].dt.month.astype(int)

# 행정구 통일
call_ratio['sub_address'] = call_ratio['sub_address'].replace({'일광면': '일광읍', '정관면': '정관읍'})

# 'cat' 비율
r1 = call_ratio.copy().groupby(['year', 'month', 'address_gu', 'sub_address', 'cat'])['call_count'].sum().reset_index()
r1_tot = call_ratio.copy().groupby(['year', 'month', 'address_gu','sub_address'])['call_count'].sum().reset_index()
r1['year'] = r1['year'] + 1
r1_tot['year'] = r1_tot['year'] + 1

r1 = r1.merge(r1_tot, how = 'left', on = ['year', 'month', 'address_gu', 'sub_address'])
r1['ratio'] = r1['call_count_x'] / r1['call_count_y']

r1 = r1.pivot(index = ['year', 'month', 'address_gu', 'sub_address'], columns = 'cat', values = 'ratio').reset_index()
r1 = r1.fillna(0)
r1.columns = ['year', 'month', 'address_gu', 'sub_address'] + [f'prev_year_{col}' for col in r1.columns[4: ]]

In [ ]:
# 바로 전 'year'의 'sub_address' 별 'sub_cat' 비율
r2 = call_ratio.copy().groupby(['year', 'month', 'address_gu', 'sub_address', 'sub_cat'])['call_count'].sum().reset_index()
r2_tot = call_ratio.copy().groupby(['year', 'month', 'address_gu','sub_address'])['call_count'].sum().reset_index()
r2['year'] = r2['year'] + 1
r2_tot['year'] = r2_tot['year'] + 1

r2 = r2.merge(r2_tot, how = 'left', on = ['year', 'month', 'address_gu', 'sub_address'])
r2['ratio'] = r2['call_count_x'] / r2['call_count_y']

r2 = r2.pivot(index = ['year', 'month', 'address_gu', 'sub_address'], columns = 'sub_cat', values = 'ratio').reset_index()
r2 = r2.fillna(0)
r2.columns = ['year', 'month', 'address_gu', 'sub_address'] + [f'prev_year_{col}' for col in r2.columns[4: ]]

r2 = r2.rename(columns = {'prev_year_기타': 'prev_year_sub_기타'})

In [ ]:
# 병합
r = pd.merge(r1, r2, 'inner', ['year', 'month', 'address_gu', 'sub_address'])

# 날짜 범위
date_range = pd.date_range(start = '2020-05-01', end = '2024-10-31', freq = 'ME')
date_df = pd.DataFrame({
    'year': date_range.year,
    'month': date_range.month
})

# Unique 'address_gu' + 'sub_address' 조합 추출
sub_addrs = r[['address_gu', 'sub_address']].drop_duplicates()
sub_addrs = sub_addrs.sort_values(['address_gu', 'sub_address']).reset_index(drop = True)

# 'year' + 'month' + 'day' + 'address_gu' + 'sub_address' 조합 생성
full_index = date_df.merge(sub_addrs, how = 'cross')
full_index = full_index.loc[full_index['month'].isin(range(5, 11))]

# 빈 날짜 + 지역 조합 채우기
r = full_index.merge(
    r,
    how = 'left',
    on = ['year', 'month', 'address_gu', 'sub_address']
)

# 결측값은 연도에 무관한 월별 중간값으로 대체
cols_to_fill = r.columns.difference(
    ['year', 'month', 'address_gu', 'sub_address']
).to_list()

r[cols_to_fill] = r[cols_to_fill].fillna(
    r.groupby(['month', 'address_gu', 'sub_address'])[cols_to_fill].transform('median')
)

# 그럼에도 불구하고 결측치가 있다면 0.0으로 대체
r.fillna(0, inplace = True)

# 회의 후 필요한 컬럼만 선택: 'sub_cat'에서는 '구급기타'만 선택
r = r.copy()[['year', 'month', 'address_gu', 'sub_address', 'prev_year_구급',
              'prev_year_구조', 'prev_year_기타', 'prev_year_화재', 'prev_year_교통사고',
              'prev_year_구급기타']]

# 저장
r.to_csv('merge9.csv', index = False, encoding = 'cp949')

# 조회
r

# 전처리

In [ ]:
# --------------------------------------------------
# Test data를 고려해 input에 자동으로 feature를 추가하는 함수 정의
# --------------------------------------------------
# ! ! ! 입력되는 dataframe은 train 데이터와 같은 dtype과 (prefix를 뗀) 컬럼명을 가져야 함 ! ! !
# --------------------------------------------------
def wowthatisamazing(x):
    out = x.copy()
    
    # -----
    # 날짜를 timestamp로 변환 후 정수 컬럼 'year', 'month', 'day', 'day_of_the_week' 생성
    # -----
    out['tm'] = pd.to_datetime(out['tm'].astype(str), format = '%Y%m%d')

    # 요일
    out['day_of_the_week'] = out['tm'].dt.day_name().astype(object)
    out['year'] = out['tm'].dt.year.astype(int)
    out['month'] = out['tm'].dt.month.astype(int)
    out['day'] = out['tm'].dt.day.astype(int)

    # -----
    # 원본 날짜 컬럼 제거
    # -----
    out = out.drop('tm', axis = 1)

    # -----
    # 기상 자료 dtype 정리
    # -----
    out[[
        'ta_max', 'ta_min', 'ta_max_min', 'hm_min', 
        'hm_max', 'ws_max', 'ws_ins_max', 'rn_day'
    ]] = out[[
        'ta_max', 'ta_min', 'ta_max_min', 'hm_min', 
        'hm_max', 'ws_max', 'ws_ins_max', 'rn_day'
    ]].astype(float)
    out['stn'] = out['stn'].astype(object)

    # -----
    # 면에서 읍으로 승격된 행정 구역 통일 ('merge2.csv'와 'merge9.csv'가 작동하기 위한 조건)
    # -----
    out['sub_address'] = out['sub_address'].replace({'일광면': '일광읍', '정관면': '정관읍'})

    merge0 = pd.read_csv('merge0.csv', encoding = 'cp949')
    merge1 = pd.read_csv('merge1.csv', encoding = 'cp949')
    merge2 = pd.read_csv('merge2.csv', encoding = 'cp949')
    
    merge4 = pd.read_csv('merge4.csv', encoding = 'cp949')
    
    merge6 = pd.read_csv('merge6.csv', encoding = 'cp949', dtype = {'gu_cluster': object})
    merge7 = pd.read_csv('merge7.csv', encoding = 'cp949')
    merge8 = pd.read_csv('merge8.csv', encoding = 'cp949')
    merge9 = pd.read_csv('merge9.csv', encoding = 'cp949')

    out = out.merge(merge0, how = 'left', on = ['year', 'month', 'address_gu'])
    out = out.merge(merge1, how = 'left', on = ['year', 'month', 'address_gu', 'sub_address'])
    out = out.merge(merge2, how = 'left', on = ['year', 'month', 'address_gu'])
    
    out = out.merge(merge4, how = 'left', on = ['year', 'month', 'day', 'address_gu', 'sub_address'])
    
    out = out.merge(merge6, how = 'left', on = ['address_gu'])
    out = out.merge(merge7, how = 'left', on = ['address_gu', 'sub_address'])
    out = out.merge(merge8, how = 'left', on = ['year', 'month', 'address_gu', 'sub_address'])
    out = out.merge(merge9, how = 'left', on = ['year', 'month', 'address_gu', 'sub_address'])

    # -----
    # 날씨 파생 변수
    # -----
    cols_to_fill = [
        'ta_max', 'ta_min', 'ta_max_min', 'hm_min',
        'hm_max', 'ws_max', 'ws_ins_max', 'rn_day'
    ]

    out2 = out.copy()[[
        'year', 'month', 'day', 'address_gu', 'sub_address', 'gu_cluster'
    ] + cols_to_fill]

    # 3단계 rolling 기반 imputation

    # 1차: 같은 날짜 + 같은 구의 과거 10일 평균으로 imputation
    for col in cols_to_fill:
        out2[col] = out2.groupby(['year', 'month', 'day', 'address_gu'])[col].transform(
            lambda s: s.fillna(s.shift(1).rolling(window = 10, min_periods = 1).mean())
        )

    # 2차: 같은 날짜 + 같은 클러스터의 과거 10일 평균으로 imputation
    for col in cols_to_fill:
        out2[col] = out2.groupby(['year', 'month', 'day', 'gu_cluster'])[col].transform(
            lambda s: s.fillna(s.shift(1).rolling(window = 10, min_periods = 1).mean())
        )

    # 3차: 같은 월 + 같은 클러스터의 과거 10일 평균으로 imputation
    for col in cols_to_fill:
        out2[col] = out2.groupby(['month', 'gu_cluster'])[col].transform(
            lambda s: s.fillna(s.shift(1).rolling(window = 10, min_periods = 1).mean())
        )
    
    # 그럼에도 결측치가 있다면 클러스터의 중간값으로 대체
    out2[cols_to_fill] = out2[cols_to_fill].fillna(out2.groupby(['year', 'month', 'day', 'gu_cluster'])[cols_to_fill].transform('median'))

    # 생성한 결측치를 원본에 넣음
    out[cols_to_fill] = out[cols_to_fill].fillna(out2[cols_to_fill])

    # 파생 변수 생성
    out['hm_range'] = out['hm_max'] - out['hm_min']
    out['ta_hm_ratio'] = out['ta_max'] / out['hm_min']
    out['wind_diff'] = out['ws_ins_max'] - out['ws_max']
    out['hot_day'] = (out['ta_max'] > 30).astype(int)
    out['humid_day'] = (out['hm_min'] > 70).astype(int)
    out['windy_day'] = ((out['ws_max'] > 10) | (out['ws_ins_max'] > 15)).astype(int)
    out['rainy_day'] = (out['rn_day'] > 0).astype(int)
    out['heavy_rain_day'] = (out['rn_day'] >= 30).astype(int)

    # 비율 inf를 max로 변경
    out['ta_hm_ratio'] = out['ta_hm_ratio'].replace(
        np.inf,
        out['ta_hm_ratio'][~np.isinf(out['ta_hm_ratio'])].max()
    )

    # 2020-2023의 날짜 + 구별 날씨 데이터 평균 (모든 날씨는 stn을 기준으로 집계되므로, 관측 지점을 섞어 새로운 변수를 만듬. 'sub_address'별로 집계 시 섞이지 않음)
    gu_cols_to_fill = [
        'gu_ta_max', 'gu_ta_min', 'gu_ta_max_min', 'gu_hm_min', 
        'gu_hm_max', 'gu_ws_max', 'gu_ws_ins_max', 'gu_rn_day'
    ]
    out[gu_cols_to_fill] = out.groupby(
        ['year', 'month', 'day', 'address_gu']
    )[cols_to_fill].transform('mean')

    # -----
    # 8월, 9월 여부
    # -----
    out['is_aug_sep'] = (out['month'].isin([8, 9])).astype(int)

    out = out.sort_values(by = ['year', 'month', 'day'], ascending = True, ignore_index = True)
    
    return out

In [ ]:
# 변수 추가
df_full = wowthatisamazing(df)

# 출력 변수
y = df_full['call_count']

# 입력 변수
x = df_full.drop(['Unnamed: 0', 'address_city', 'call_count'], axis = 1)

# 컬럼 타입 구분
num_col = x.select_dtypes(include = 'number').columns.to_list()
cat_col = x.select_dtypes(exclude = 'number').columns.to_list()

# 전처리기
preprocessor = ColumnTransformer([
    ('num', 'passthrough', num_col),
    ('cat', TargetEncoder(
        target_type = 'continuous', 
        shuffle = False
    ), cat_col)
]).set_output(transform = 'pandas')

# CV 정의 
cv = TimeSeriesSplit(n_splits = 5)

In [ ]:
# Test 데이터
test = pd.read_csv('test_call119.csv', encoding = 'cp949')

# 컬럼명 정리
x_test = test.copy().rename(
    columns = {'TM': 'tm',
               'STN': 'stn'}
)

# Feature 추가
x_test = wowthatisamazing(x_test)

# 불필요한 컬럼 정리
x_test = x_test.drop(['address_city', 'call_count'], axis = 1)

# LGBM

### 모형 선택 (LGBM)

In [ ]:
# 모형 파이프라인
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LGBMRegressor(
        objective = 'regression',
        subsample_freq = 1, 
        random_state = 42, 
        verbosity = -1, 
        device = 'gpu'
    ))
])

In [ ]:
# Optuna 목적 함수
def objective(trial):
    params = {
        'model__num_leaves': trial.suggest_int('num_leaves', 31, 256),
        'model__max_depth': trial.suggest_int('max_depth', 4, 16),
        'model__learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.2, log = True),
        'model__n_estimators': trial.suggest_int('n_estimators', 300, 2000),
        'model__min_split_gain': trial.suggest_float('min_split_gain', 0.0, 0.1),
        'model__min_child_samples': trial.suggest_int('min_child_samples', 50, 300),
        'model__subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'model__colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'model__reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
        'model__reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10.0)
    }

    optuna_pipeline = clone(pipeline).set_params(**params)

    scores = cross_validate(
        optuna_pipeline, x, y,
        scoring = 'neg_root_mean_squared_error',
        cv = cv,
        n_jobs = 4,
        verbose = 1
    )
    
    return -scores['test_score'].mean()

# Optuna 실행
os.environ['PYTHONHASHSEED'] = str(42)
sampler = optuna.samplers.TPESampler(seed = 42)
study = optuna.create_study(
    direction = 'minimize',
    study_name = 'predict_call_count',
    sampler = sampler
)
study.optimize(objective, n_trials = 100, n_jobs = 3, show_progress_bar = True)

In [ ]:
# Optuna 결과
print('Best parameters:')
print(study.best_params)
print('Best RMSE:')
print(study.best_trial.value)

# 최적 파라미터 저장
with open('best_params.json', 'w') as f:
    json.dump({f'model__{k}': v for k, v in study.best_trial.params.items()}, f, indent = 4)

# 최적 RMSE 저장
with open('best_rmse.txt', 'w') as f:
    f.write(str(study.best_trial.value))

In [ ]:
# 전체 데이터로 재학습
with open('best_params.json', 'r') as f:
    best_params = json.load(f)

best_lgbm = pipeline.set_params(**best_params)
best_lgbm = best_lgbm.fit(x, y)

# RF

### 모형 선택 (RF)

In [ ]:
# 모형 파이프라인
pipeline2 = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LGBMRegressor(
        boosting_type = 'rf',
        learning_rate = 1.0,
        objective = 'regression',
        bagging_freq = 1,
        min_child_samples = 1,
        random_state = 42, 
        boost_from_average = False,
        verbosity = -1, 
        device = 'gpu'
    ))
])

In [ ]:
# Optuna 목적 함수
def objective2(trial):
    params = {
        'model__num_leaves': trial.suggest_int('num_leaves', 128, 256),
        'model__n_estimators': trial.suggest_int('n_estimators', 300, 1000),
        'model__bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 0.9),
        'model__feature_fraction': trial.suggest_float('feature_fraction', 0.6, 0.9),
    }

    optuna_pipeline = clone(pipeline2).set_params(**params)

    scores = cross_validate(
        optuna_pipeline, x, y,
        scoring = 'neg_root_mean_squared_error',
        cv = cv,
        n_jobs = 4,
        verbose = 1
    )
    
    return -scores['test_score'].mean()

# Optuna 실행
os.environ['PYTHONHASHSEED'] = str(42)
sampler2 = optuna.samplers.TPESampler(seed = 42)
study2 = optuna.create_study(
    direction = 'minimize',
    study_name = 'predict_call_count2',
    sampler = sampler2
)
study2.optimize(objective2, n_trials = 50, n_jobs = 3, show_progress_bar = True)

In [ ]:
# Optuna 결과
print('Best parameters:')
print(study2.best_params)
print('Best RMSE:')
print(study2.best_trial.value)

# 최적 파라미터 저장
with open('best_params2.json', 'w') as f:
    json.dump({f'model__{k}': v for k, v in study2.best_trial.params.items()}, f, indent = 4)

# 최적 RMSE 저장
with open('best_rmse2.txt', 'w') as f:
    f.write(str(study2.best_trial.value))

In [ ]:
# 전체 데이터로 재학습
with open('best_params2.json', 'r') as f:
    best_params2 = json.load(f)

best_rf = pipeline2.set_params(**best_params2)
best_rf = best_rf.fit(x, y)

# XGB

### 모형 선택 (XGB)

In [ ]:
# 모형 파이프라인
pipeline3 = Pipeline([
    ('preprocessor', preprocessor),
    ('model', XGBRegressor(
        verbosity = 1,
        objective = 'reg:squarederror',
        random_state = 42,
        tree_method = 'hist',
        device = 'cuda'
    ))
])

In [ ]:
# Optuna 목적 함수
def objective3(trial):
    params = {
        'model__n_estimators': trial.suggest_int('n_estimators', 300, 2000),
        'model__max_depth': trial.suggest_int('max_depth', 4, 16),
        'model__min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
        'model__gamma': trial.suggest_float('gamma', 0, 10.0),
        'model__learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.2, log = True),
        'model__subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'model__colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'model__reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
        'model__reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10.0)
    }

    optuna_pipeline = clone(pipeline3).set_params(**params)

    scores = cross_validate(
        optuna_pipeline, x, y,
        scoring = 'neg_root_mean_squared_error',
        cv = cv,
        n_jobs = 3,
        verbose = 1
    )
    
    return -scores['test_score'].mean()

# Optuna 실행
os.environ['PYTHONHASHSEED'] = str(42)
sampler3 = optuna.samplers.TPESampler(seed = 42)
study3 = optuna.create_study(
    direction = 'minimize',
    study_name = 'predict_call_count3',
    sampler = sampler3
)
study3.optimize(objective3, n_trials = 100, n_jobs = 3, show_progress_bar = True)

In [ ]:
# Optuna 결과
print('Best parameters:')
print(study3.best_params)
print('Best RMSE:')
print(study3.best_trial.value)

# 최적 파라미터 저장
with open('best_params3.json', 'w') as f:
    json.dump({f'model__{k}': v for k, v in study3.best_trial.params.items()}, f, indent = 4)

# 최적 RMSE 저장
with open('best_rmse3.txt', 'w') as f:
    f.write(str(study3.best_trial.value))

In [ ]:
# 전체 데이터로 재학습
with open('best_params3.json', 'r') as f:
    best_params3 = json.load(f)

best_xgb = pipeline3.set_params(**best_params3)
best_xgb = best_xgb.fit(x, y)

# 앙상블

In [ ]:
# 찾아낸 최적 모형들의 예측값으로 ridge regression (메타 모형): CV RMSE를 최소화하는 regularization parameter alpha 탐색
from sklearn.linear_model import RidgeCV

# RidgeCV
alpha_grid = np.logspace(-3, 2, 20)
ridge_cv = RidgeCV(alphas = alpha_grid, cv = cv)

# StackingRegressor를 위해 XGBoost의 device를 변경 (안 그러면 warning 나옴)
best_xgb.set_params(model__device = 'cpu')

# Base estimators
estimators = [
    ('lgbm', best_lgbm),
    ('rf', best_rf),
    ('xgb', best_xgb)
]

# Stacking 모델
stacked_model = StackingRegressor(
    estimators = estimators,
    final_estimator = ridge_cv,
    cv = 'prefit', # 'prefit': 원래는 매 fold를 학습하고 validate해야 하지만, 너무 오래 걸리므로 leakage를 허용
    passthrough = False,
    verbose = 1
)

# 학습
stacked_model.fit(x, y)

# 최적 alpha
print(f'Selected alpha: {stacked_model.final_estimator_.alpha_}')
print(f'Coefficients:{stacked_model.final_estimator_.coef_}')

In [ ]:
# 비교를 위한 CV RMSE 계산
meta_features = np.column_stack([
    best_lgbm.predict(x),
    best_rf.predict(x),
    best_xgb.predict(x)
])
scores = cross_validate(ridge_cv, meta_features, y, scoring='neg_root_mean_squared_error', cv = cv)

print("Ensemble RMSE:", -scores['test_score'].mean())

In [ ]:
# 예측
submission = stacked_model.predict(x_test)
submission = np.clip(np.round(submission), 1, None).astype(int)

# 저장
test['call_count'] = submission
test.to_csv('250259.csv', index = False, encoding = 'cp949')